In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np

import analyze

In [ ]:
sim_name = "cst"
sim_path = Path.cwd() / "src" / "sim" / sim_name

nf2ff = analyze.read_nf2ff(sim_path / "nf2ff.h5")
# fig = plt.figure(figsize=(12, 5))
# analyze.plot_ff_2d(nf2ff, ax=fig.add_subplot(121))
# analyze.plot_ff_3d(nf2ff, logscale=-20, ax=fig.add_subplot(122, projection="3d"))

In [ ]:
theta, phi = nf2ff["theta"], nf2ff["phi"]
E_norm, Dmax = nf2ff["E_norm"], nf2ff["Dmax"]
E_norm = E_norm[0]  # single frequency

analyze.plot_ff_polar(E_norm, Dmax, theta, phi, title="title")


In [ ]:
xn = 1
yn = 1
dx = 60
dy = 60
freq = 2.45e9
AF = analyze.array_factor(theta, phi, xn, yn, dx, dy, freq)

E_norm_array = E_norm * AF.T
title = f"{xn}x{yn} array pattern, {dx}x{dy} mm spacing"
analyze.plot_ff_polar(E_norm_array, Dmax, theta, phi, title=title)